In [2]:
!jupyter nbconvert --to script 'graph_classification_tutorial.ipynb'

[NbConvertApp] Converting notebook graph_classification_tutorial.ipynb to script
[NbConvertApp] Writing 5041 bytes to graph_classification_tutorial.py


In [4]:
import boto3
import re

import os
import numpy as np
from sagemaker import get_execution_role

import sagemaker as sage
from time import gmtime, strftime

In [ ]:
# S3 prefix
prefix = 'dgl_example'

# Setup session
sess = sage.Session()

# Get Sagemaker role
role = 'arn:aws:iam::397262719838:role/service-role/AmazonSageMaker-ExecutionRole-20171213T134005'

CODE_PATH = 'graph_classification_tutorial.py'
code_location = sess.upload_data(CODE_PATH, key_prefix=prefix)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-graph-class:latest'.format(account, region)
print(image)

params = {}
params['entrypoint'] = CODE_PATH
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       hyperparameters=params,
                       output_path="s3://dgl-internal-example/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

data = {}
data['training-code'] = code_location
data['training-data'] = 's3://dgl-sagemaker-dataset/dgl/'

tree.fit(data)